In [ ]:
%matplotlib inline
import sklearn
import numpy as np
import matplotlib.pyplot as plt

import librosa
import librosa.display

import IPython.display as ipd

import warnings
warnings.filterwarnings('ignore')

import import_ipynb
import preprocessor
import display
import classifier
import csv_file_io
import datetime

PROJECT_HOME = './'
# DRONE = 'Inspire2' # 'Bebop2', 'Spark', 'MavicPro', 'Phantom4', 'Matrice100', 'MavicAir', 'AutelEVO', 'Inspire2', 'JME', 'ParrotAnafi']
# DATASET_DIR = PROJECT_HOME + 'dataset/validation/Inspire2/Inspire2_validation_File3.wav'
RESULT_FILE_DIR = PROJECT_HOME + 'results/'
DATASET_FILE_DIR = PROJECT_HOME + 'results/dataset/'
TRAINING_DATASET_DIR = PROJECT_HOME + 'dataset/1. training/'
VALIDATION_DATASET_DIR = PROJECT_HOME + 'dataset/2. validation/'
TEST_DATASET_DIR = PROJECT_HOME + 'dataset/3. test/'
TRAINING_VALIDATION_DATASET_DIR = PROJECT_HOME + 'dataset/4. training_validation/'
TRAINING_VALIDATION_TEST_DATASET_DIR = PROJECT_HOME + 'dataset/5. training_validation_test/'
SEGMENTATION_DATASET_DIR = PROJECT_HOME + 'dataset/6. segmentation/'

now = datetime.datetime.now()
now_date_time = now.strftime('%Y%m%d_%H%M%S')[2:]

RESULT_FILE_NAME = "[Training+Validation+Test] Classification result ()"
DATASET_FILE_NAME = "Dataset "

SAMPLE_RATE = 44100
FIG_SIZE = (15,5)

FRAME_LENGTH = 0.250 # 250 ms
FRAME_STRIDE = 0.250 # same with frame_length for 0% overlap

DIGITS = 3

b_visualizing = False

def save_each_classification_performance(kernel: str, target: list, acc: float, tpr: float, fnr: float, tnr: float, fpr: float):
    file_name = RESULT_FILE_NAME.split(')')[0] + kernel + ') '

    f = csv_file_io.open_csv(RESULT_FILE_DIR + file_name + now_date_time + '.csv', 'w')
    csv_file_io.write_csv_data(f, ['Kernel', kernel])
    csv_file_io.write_csv_data(f, ['ACC', acc])
    csv_file_io.write_csv_data(f, ['', 'TPR', 'FNR', 'TNR', 'FPR'])
    
    for i in range (0, len(target)):
        csv_file_io.write_csv_data(f, [target[i], tpr[i], fnr[i], tnr[i], fpr[i]])    

    csv_file_io.write_csv_data(f, [])
    csv_file_io.write_csv_data(f, ['Average', round(np.mean(tpr), DIGITS), round(np.mean(fnr), DIGITS), round(np.mean(tnr), DIGITS), round(np.mean(fpr), DIGITS)])

    f.close()

    # f = csv_file_io.open_csv(RESULT_FILE_DIR + file_name + now_date_time + '.csv', 'a')
    # csv_file_io.write_csv_data(f, [kernel, acc, tpr, fnr, tnr, fpr])
    # f.close()


def save_total_classification_performance(res_dict: dict):
    file_name = RESULT_FILE_NAME.split(')')[0] + 'total) '
    f = csv_file_io.open_csv(RESULT_FILE_DIR + file_name + now_date_time + '.csv', 'w')
    csv_file_io.write_csv_data(f, ['Kernel', 'ACC', 'TPR', 'FNR', 'TNR', 'FPR'])
    # f.close()

    for i in range(0, len(res_dict['kernel'])):
        # f = csv_file_io.open_csv(RESULT_FILE_DIR + FILE_NAME + now_date_time + '.csv', 'a')
        csv_file_io.write_csv_data(f, [res_dict['kernel'][i], res_dict['acc'][i],  res_dict['tpr'][i], res_dict['fnr'][i], res_dict['tnr'][i], res_dict['fpr'][i]])    

    f.close()


In [ ]:
dict_integrated_train_validation_test_signal = preprocessor.get_integrated_signal(SEGMENTATION_DATASET_DIR, type='SEGMENTATION')

# Visualizing Audio (waveform and power spectrum)
if b_visualizing == True:
    for key, val in dict_integrated_train_validation_test_signal.items():
        display.display_waveshow(val, key + "'s Training+Validation+Test Signal's Waveform")
        display.display_power_spectrum(val, key + "'s Training+Validation+Test Signal's Power Spectrum")

In [ ]:
# Feature Extraction (MFCC)
N_MFCC = 40
n_fft = int(SAMPLE_RATE * FRAME_LENGTH) # length of the FFT window, frame 하나당 sample의 수 / n_fft=2048, n_fft = int(SAMPLE_RATE * FRAME_LENGTH)
hop_length = int(SAMPLE_RATE * FRAME_STRIDE) # number of samples between successive frames / hop_length=512, hop_length = int(SAMPLE_RATE * FRAME_STRIDE)
n_mels = 128
f_max = 8000

dict_melspectrogram_train_validation_test_signal = {}
for key, val in dict_integrated_train_validation_test_signal.items():    
    dict_melspectrogram_train_validation_test_signal[key] = librosa.feature.melspectrogram(val, sr=SAMPLE_RATE, n_mels=n_mels, fmax=f_max, n_fft=n_fft, hop_length=hop_length)    

dict_mfccs_train_validation_test_signal = {}
for key, val in dict_melspectrogram_train_validation_test_signal.items():
    dict_mfccs_train_validation_test_signal[key] = librosa.feature.mfcc(S=librosa.power_to_db(val), sr=SAMPLE_RATE, n_mfcc=N_MFCC)

# print(dict_melspectrogram_train_validation_test_signal['AutelEVO'].shape)
# print(dict_mfccs_train_validation_test_signal['AutelEVO'].shape)

if b_visualizing == True:
    for key, val in dict_mfccs_train_validation_test_signal.items():
        display.display_mfccs(val,  "'s Training+Validation+Test Signal's MFCCs")  

In [ ]:
arr_train_validation_test_data = None
arr_train_validation_test_target = None

for key, val in dict_mfccs_train_validation_test_signal.items():    
    arr_mfccs_data = val.transpose()    
    arr_target = np.full((arr_mfccs_data.shape[0], 1), key)   

    if arr_train_validation_test_data is None:
        arr_train_validation_test_data = arr_mfccs_data
        arr_train_validation_test_target = arr_target        
    else:       
        arr_train_validation_test_data = np.vstack((arr_train_validation_test_data, arr_mfccs_data))
        arr_train_validation_test_target = np.vstack((arr_train_validation_test_target, arr_target))

print("Training+Validation+Test data / target : ", arr_train_validation_test_data.shape, arr_train_validation_test_target.shape)
ararr_train_validation_test_data_target = np.hstack((arr_train_validation_test_data, arr_train_validation_test_target))
# np.savetxt(DATASET_FILE_DIR + DATASET_FILE_NAME + now_date_time + '.csv', ararr_train_validation_test_data_target, delimiter=",", fmt='%s')

In [ ]:
# arr_std_train_validation_test_data = classifier.set_scaler_train(arr_train_validation_test_data)

CV = 10
b_standardscaler = False

classifier.cross_validation_svm("linear", CV, arr_train_validation_test_data, arr_train_validation_test_target, b_standardscaler)
classifier.cross_validation_svm("LinearSVC", CV, arr_train_validation_test_data, arr_train_validation_test_target, b_standardscaler)
classifier.cross_validation_svm("rbf", CV, arr_train_validation_test_data, arr_train_validation_test_target, b_standardscaler)
classifier.cross_validation_svm("poly", CV, arr_train_validation_test_data, arr_train_validation_test_target, b_standardscaler)
classifier.cross_validation_svm("sigmoid", CV, arr_train_validation_test_data, arr_train_validation_test_target, b_standardscaler)

In [ ]:
b_standardscaler = True

classifier.cross_validation_svm("linear", CV, arr_train_validation_test_data, arr_train_validation_test_target, b_standardscaler)
classifier.cross_validation_svm("LinearSVC", CV, arr_train_validation_test_data, arr_train_validation_test_target, b_standardscaler)
classifier.cross_validation_svm("rbf", CV, arr_train_validation_test_data, arr_train_validation_test_target, b_standardscaler)
classifier.cross_validation_svm("poly", CV, arr_train_validation_test_data, arr_train_validation_test_target, b_standardscaler)
classifier.cross_validation_svm("sigmoid", CV, arr_train_validation_test_data, arr_train_validation_test_target, b_standardscaler)

In [ ]:
b_standardscaler = True
CV = 10

list_C = np.arange(0.01, 5.01, 0.10)
PARAMETERS = {'C': list_C} # PARAMETERS = {'C': [0.001, 0.01, 0.1, 1, 10, 25, 50, 100]}
classifier.gridsearchCV_svm("linear", CV, PARAMETERS, arr_train_validation_test_data, arr_train_validation_test_target, b_standardscaler)
classifier.gridsearchCV_svm("LinearSVC", CV, PARAMETERS, arr_train_validation_test_data, arr_train_validation_test_target, b_standardscaler)

# gamma: rbf, poly, sigmoid / degree: poly
list_C = np.arange(5.0, 31.0, 1.0)
list_gamma = np.arange(0.005, 0.016, 0.001)
PARAMETERS = {'C': list_C, 'gamma': list_gamma} # PARAMETERS = {'C': [0.001, 0.01, 0.1, 1, 10, 25, 50, 100], 'gamma':[0.001, 0.01, 0.1, 1, 10, 25, 50, 100]}
classifier.gridsearchCV_svm("rbf", CV, PARAMETERS, arr_train_validation_test_data, arr_train_validation_test_target, b_standardscaler)

list_C = np.arange(20.0, 71.0, 1.0)
list_gamma = np.arange(0.005, 0.016, 0.001)
PARAMETERS = {'C': list_C, 'gamma': list_gamma}
classifier.gridsearchCV_svm("poly", CV, PARAMETERS, arr_train_validation_test_data, arr_train_validation_test_target, b_standardscaler)

list_C = np.arange(20.0, 71.0, 1.0)
list_gamma = np.arange(0.0005, 0.0016, 0.0001)
PARAMETERS = {'C': list_C, 'gamma': list_gamma}
classifier.gridsearchCV_svm("sigmoid", CV, PARAMETERS, arr_train_validation_test_data, arr_train_validation_test_target, b_standardscaler)

In [ ]:
b_standardscaler = False
CV = 10

list_C = np.arange(0.01, 5.01, 0.10)
PARAMETERS = {'C': list_C}
classifier.gridsearchCV_svm("linear", CV, PARAMETERS, arr_train_validation_test_data, arr_train_validation_test_target, b_standardscaler)
classifier.gridsearchCV_svm("LinearSVC", CV, PARAMETERS, arr_train_validation_test_data, arr_train_validation_test_target, b_standardscaler)

# gamma: rbf, poly, sigmoid / degree: poly
list_C = np.arange(5.0, 31.0, 1.0)
list_gamma = np.arange(0.005, 0.016, 0.001)
PARAMETERS = {'C': list_C, 'gamma': list_gamma}
classifier.gridsearchCV_svm("rbf", CV, PARAMETERS, arr_train_validation_test_data, arr_train_validation_test_target, b_standardscaler)

list_C = np.arange(20.0, 71.0, 1.0)
list_gamma = np.arange(0.005, 0.016, 0.001)
PARAMETERS = {'C': list_C, 'gamma': list_gamma}
classifier.gridsearchCV_svm("poly", CV, PARAMETERS, arr_train_validation_test_data, arr_train_validation_test_target, b_standardscaler)

list_C = np.arange(20.0, 71.0, 1.0)
list_gamma = np.arange(0.0005, 0.0016, 0.0001)
PARAMETERS = {'C': list_C, 'gamma': list_gamma}
classifier.gridsearchCV_svm("sigmoid", CV, PARAMETERS, arr_train_validation_test_data, arr_train_validation_test_target, b_standardscaler)

In [ ]:
CV = 10

# gamma: rbf, poly, sigmoid / degree: poly
list_C = np.arange(5.0, 31.0, 1.0)
list_gamma = np.arange(0.005, 0.016, 0.001)
PARAMETERS = {'C': list_C, 'gamma': list_gamma}
# PARAMETERS = {'C': [0.001, 0.01, 0.1, 1, 10, 25, 50, 100],\
#               'gamma':[0.001, 0.01, 0.1, 1, 10, 25, 50, 100]}

classifier.gridsearchCV_svm("rbf", CV, PARAMETERS, arr_train_validation_test_data, arr_train_validation_test_target, b_standardscaler)

In [ ]:
CV = 10

list_C = np.arange(20.0, 71.0, 1.0)
list_gamma = np.arange(0.005, 0.016, 0.001)
PARAMETERS = {'C': list_C, 'gamma': list_gamma}

classifier.gridsearchCV_svm("poly", CV, PARAMETERS, arr_train_validation_test_data, arr_train_validation_test_target, b_standardscaler)

In [ ]:
CV = 10

list_C = np.arange(20.0, 71.0, 1.0)
list_gamma = np.arange(0.0005, 0.0016, 0.0001)
PARAMETERS = {'C': list_C, 'gamma': list_gamma}

classifier.gridsearchCV_svm("sigmoid", CV, PARAMETERS, arr_train_validation_test_data, arr_train_validation_test_target, b_standardscaler)